In [2]:
#구글드라이브와 colab 파일 연결

from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [22]:
import numpy as np
import matplotlib.pyplot as pylab
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from skimage.io import imread
from skimage import img_as_float
import matplotlib.image as mpimg

import matplotlib.pylab as pylab
from skimage import img_as_float

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline #얼굴 이미지 처리를 위해 필요한 모듈들 


def LoadFaceimg(): #얼굴 이미지를 불러오는 함수.
#0001_0001~0350_0005를 faces[0]~faces[1749] 로 불러옴.
#따라서 faces[index] 에서 사람 번호인 personID = (index // 5 + 1),
#사진 번호를 photoID라 하면 photoID = (index % 5 + 1) 와 같다.

  #1~9번 사람은 그 사람의 ID인 사진 파일 이름이 0001~0009,
  #10~99번은 0011~0099, 100~350번은 0100~0350 으로 되어있어
  #문자열을 맞추기 위해 ID가 한자릿수, 십자릿수, 백자릿수를 최대로 하는 것에 따라
  #for문을 따로 이용했다.

  faces = [] #face 배열 선언하여 얼굴 이미지들을 순서대로 삽입

  for i in range(1,10): #일의 자릿수 ID
    for j in range(1,6): 
      img = img_as_float(imread('/gdrive/My Drive/ColabNote/ARFaces/mpeg_000'+str(i)+'_000'+str(j)+'.BMP'))
      faces.append(img)

  for i in range(10,100): #십의 자릿수 ID
    for j in range(1,6): 
      img = img_as_float(imread('/gdrive/My Drive/ColabNote/ARFaces/mpeg_00'+str(i)+'_000'+str(j)+'.BMP'))
      faces.append(img)

  for i in range(100, 351): #백의 자릿수 ID
    for j in range(1,6): 
      img = img_as_float(imread('/gdrive/My Drive/ColabNote/ARFaces/mpeg_0'+str(i)+'_000'+str(j)+'.BMP'))
      faces.append(img)     

  faces = np.array(faces) #face에 모두 넣었으므로, 적절한 연산을 위해 numpy 배열로 바꾼다.
  faces = faces.reshape((1750, 56*46))
  
  return faces




#특징벡터의 start~end 열만을 계산하는 함수.
def projection(faces_train, faces_test, start, end):
    #identification
  #(1) 각 얼굴에 대한 특징벡터를 계산
  #(2) 특징벡터 간의 거리를 계산
  #(3) 거리가 가장 작은 4개의 얼굴은 동일 ID로 취급하여
  #(4) mr 계산

  #가장 적합한 직선을 찾는 fit
  pca = PCA(0.8) #0.9일때는 주성분 개수가 51개로 너무 많아 0.8을 이용했다.
  pca.fit(faces_train)
  n_comp = pca.n_components_ #16
  #0.7일때는 8

  #그 직선에 투영하는 transform
  faces_proj = pca.transform(faces_train)
  faces_test_proj = pca.transform(faces_test) #test data는 fit 단계를 거치지 않고 투영시킴

  new_faces_proj = []
  new_faces_test_proj = []

  for i in range(1400):
    for j in range(start-1,end): 
      new_faces_proj.append(faces_proj[i][j])
 
  for i in range(350):
    for j in range(start-1,end): 
      new_faces_test_proj.append(faces_test_proj[i][j])

  new_faces_proj = np.array(new_faces_proj)
  new_faces_proj = new_faces_proj.reshape(1400, end-start+1)

  new_faces_test_proj = np.array(new_faces_test_proj)
  new_faces_test_proj = new_faces_test_proj.reshape(350, end-start+1)

  return new_faces_proj, new_faces_test_proj



def print_distance(faces_train, faces_test, start,end): #1~10 ID를 가진 얼굴 간의 거리를 출력해주는 함수
  #identification
  #(1) 각 얼굴에 대한 특징벡터를 계산
  #(2) 특징벡터 간의 거리를 계산
  
  faces_proj, faces_test_proj = projection(faces_train, faces_test, start,end) #faces_proj, faces_test_proj을 가져옴.

  #특징벡터간의 거리 계산
  distance = np.zeros((10,10*4))

  for i in range(0,5): #test
    for j in range(0, 5*4): #train
      distance[i][j] = np.sqrt(np.sum((faces_test_proj[i]-faces_proj[j])**2)) #거리는 유클리드 거리를 이용한다.
      #distance[i][j] 에는, (i+1)번째 사람의 1번째 얼굴과 (j//4 + 1))번째 사람의 (j%4 + 2) 번째 얼굴 사이의 거리가 담겨있음.

  distance = np.array(distance)

  print('table', end=' ')
  for j in range(0,4*5):
    print("{:>5}".format('{}'.format(j//4 + 1)), end=' ')

  for i in range(0,5):
    print('\n')
    print(i+1, end= '     ')
    for j in range(0, 5*4):
      print("{:>5}".format('{}'.format(round(distance[i][j], 2))), end = ' ')




#training set과 test set을 넣으면 mr(identification)을 계산하는 함수.
def identification(faces_train, faces_test, start, end): 

  faces_proj, faces_test_proj = projection(faces_train, faces_test, start,end) #faces_proj, faces_test_proj을 가져옴.

  #특징벡터간의 거리 계산
  distance = np.zeros((350,350*4))

  for i in range(0,350): #test
    for j in range(0, 350*4): #train
      distance[i][j] = np.sqrt(np.sum((faces_test_proj[i]-faces_proj[j])**2)) #거리는 유클리드 거리를 이용한다.
      #distance[i][j] 에는, (i+1)번째 사람의 1번째 얼굴과 (j//4 + 1))번째 사람의 (j%4 + 2) 번째 얼굴 사이의 거리가 담겨있음.

  distance = np.array(distance)


  min_index = 0
  min_index_array = np.zeros((350,4))
  dist_compare = np.zeros(350)

  for i in range(350): #350개의 test data에 대해 1400개의 training data와 비교
    dist_compare = distance[i].copy()
    for k in range(4): #거리가 가장 작은 얼굴 4개를 찾기 위해 4번 반복
      min = 500           #거리가 가장 작은 얼굴 4개를 찾기 위해 min 값을 distance 값에 존재하지 않는 500으로 설정
      for j in range(350*4): #1400개의 training data와 비교
        if(min > dist_compare[j]): 
          min = dist_compare[j]
          min_index = j

      min_index_array[i][k] = min_index // 4 # 가장 작은 거리값의 id (0~349) 를 저장
      #배열 index 특성상 1~350인 ID가 0~349가 되었다.
      dist_compare[min_index] = 500 
      #가장 작은 거리값을 4개 찾아야 하므로, 
      #현재 가장 작은 값을 가지는 곳이 또 나오면 안됨. 그 곳의 값을 500으로 변경.

  #따라서 min_index_array[i][k]에는 i번 사람의 test data와 1400개의 training data 사이의
  #거리가 가장 가까운 순으로 4개의 data가 min_index_array[i][0]~min_index_array[i][3]에 담겨있다.

  np.set_printoptions(suppress=True)
  #print(min_index_array)


  #matching_rate 계산
  mr = np.zeros(350)
  mr1count = 0

  #거리가 가장 작은 4개의 얼굴을 같은 ID로 인식 했을 때
  for i in range(350):
    mr4count = 0
    for j in range(4):
      if(i == min_index_array[i][j]): #ID가 같으면 count를 증가시킴.
        mr4count += 1 
    mr[i] = 100 * mr4count / 4    #test data 각각에 대해 4개 중 몇개의 ID가 일치하는지 퍼센트로 나타냄.

  mr4_result = 0
  for i in range(350):
    mr4_result += mr[i] / 350    #위에서 350개에 test data에 대해 각각 구한 일치비율의 평균을 낸다.

  return mr4_result

In [5]:
#먼저 원래 mr값을 알기 위해 원래 이미지에 대해
#training set, test set을 나누어 계산하여 mr값을 구한다.
#mr값을 구하는 과정은 함수로 만들어두었다.
faces = LoadFaceimg() #이미지 불러오기.
#faces = faces.reshape((1750, 56*46)) # 1750개의 이미지를 각각 하나의 배열로 생각한다.

#training set과 test set 나누기
faces_train = [] #학습데이터
faces_test = [] #테스트데이터


#한 사람당 얼굴 데이터가 5개씩 있었으므로,
for i in range(0,1750):
  #photoID = (index % 5 - 1) 이므로, i%5 == 0 이면, 각 사람의 1번 얼굴
  if( i%5 == 0 ): #n번째 사람의 1번 data는 test data로
    faces_test.append(faces[i])
  else: #2~5번 data는 training data로 사용
    faces_train.append(faces[i])

faces_train = np.array(faces_train)
faces_test = np.array(faces_test)
#print(len(faces_train), len(faces_test), len(faces_train)+len(faces_test))
print('original', end= ' ')
print(identification(faces_train, faces_test, 1,16), end = ' ') #original이므로 start=1, end=16 을 이용
print(faces.shape) #(1750, 56*46) -> (56*46) 화소의 400개의 얼굴 영상





#특징벡터의 열을 줄인 것의 mr을 전부 출력해본다.
print('minus')

for i in range(1,16): #몇 개를 넣을지 결정.
  print(str(i)+'개의 원소를 가진 집합에서의 mr:', end=' ')
  for j in range(16-i+1): #원소가 1개라면, 16개의 집합, 원소가 2개라면 연속된 15개의 집합 ... 원소가 i개라면 연속된 17-i개의 집합이 생김. 
    start = j + 1 #start부터
    end = start + i - 1 #end (start+넣을개수-1)까지.

    print(str(start)+'~'+str(end)+' '+str(round(identification(faces_train, faces_test, start,end),2)), end = '  ') #소숫점 둘째자리에서 반올림
  print('\n')

'''
print('16개의 원소를 가진 집합에서의 mr:', end=' ')
print(str(1)+'~'+str(16)+' '+str(round(identification(faces_train, faces_test, 1,16),2)), end = '  ') #original과 같음.
'''

original 47.2142857142858 (1750, 2576)
minus
1개의 원소를 가진 집합에서의 mr: 1~1 2.21  2~2 3.0  3~3 0.79  4~4 1.71  5~5 1.43  6~6 0.64  7~7 1.21  8~8 1.57  9~9 1.86  10~10 1.93  11~11 0.43  12~12 0.93  13~13 1.14  14~14 0.86  15~15 0.64  16~16 0.79  

2개의 원소를 가진 집합에서의 mr: 1~2 13.29  2~3 8.71  3~4 5.36  4~5 6.36  5~6 4.64  6~7 4.29  7~8 5.79  8~9 5.14  9~10 5.86  10~11 3.93  11~12 2.5  12~13 3.64  13~14 3.07  14~15 2.0  15~16 2.57  

3개의 원소를 가진 집합에서의 mr: 1~3 19.64  2~4 17.0  3~5 13.86  4~6 14.43  5~7 13.21  6~8 12.57  7~9 14.36  8~10 13.43  9~11 8.71  10~12 8.79  11~13 6.36  12~14 6.29  13~15 5.79  14~16 5.93  

4개의 원소를 가진 집합에서의 mr: 1~4 29.57  2~5 25.14  3~6 19.43  4~7 21.64  5~8 21.07  6~9 19.43  7~10 22.79  8~11 19.29  9~12 15.29  10~13 14.29  11~14 11.07  12~15 11.64  13~16 10.07  

5개의 원소를 가진 집합에서의 mr: 1~5 35.93  2~6 28.86  3~7 26.64  4~8 28.0  5~9 26.36  6~10 26.0  7~11 27.14  8~12 25.14  9~13 20.79  10~14 18.86  11~15 14.93  12~16 15.64  

6개의 원소를 가진 집합에서의 mr: 1~6 38.57  2~7 34.29  3~8 32.43

In [9]:
#소숫점 2개까지는 같은 것들을 비교해보려고 프린트해봄.
print('14개 1번 시작'+str(identification(faces_train, faces_test, 1,14)))
print('14개 1번 시작'+str(identification(faces_train, faces_test, 1,16))) 

14개 1번 시작47.21428571428579
14개 1번 시작47.2142857142858


In [28]:
#실험한 data 중 가장 mr값이 작은 n값과 원래 이미지에 대한 distance 배열을 처음 10명에 대해 출력

#print_distance(faces_train, faces_test, 2,16)
#print('\n\n')
#print_distance(faces_train, faces_test, 3,15)
#print('\n\n')
print_distance(faces_train, faces_test, 3,10)
print('\n\n')
print_distance(faces_train, faces_test, 1,16) #원래 이미지에 대한 distance #mr 47.21
print('\n\n')
print_distance(faces_train, faces_test,1,12) #mr 47.36
print('\n\n')
print_distance(faces_train, faces_test,1,13) #mr 47.29
print('\n\n')

table     1     1     1     1     2     2     2     2     3     3     3     3     4     4     4     4     5     5     5     5 

1      4.61  1.76   8.7  8.81  12.1 10.47 13.48 14.07  6.99  7.24 12.56 12.14 11.23 12.58 12.73  12.0  9.53  9.76 10.12 11.04 

2      9.45  10.8  9.38 10.04  2.63  2.69  8.16  7.98  10.9  9.71 12.07 12.34  6.95   9.3  6.96  7.81  7.29  6.86  6.47  7.76 

3      5.24  6.61  6.68  8.94 10.76  9.94  8.76 10.98  2.64  2.62  6.52  8.06 11.33 13.04  10.7 10.61  7.62  7.96  5.99  8.46 

4      9.77 10.73 11.13  11.6  6.36  5.29 10.76 11.18 12.04 10.26 13.71 14.04  1.96  4.37  5.42  6.69  5.31  4.69  8.02  9.06 

5      4.92  6.95  7.44  7.93  8.23  7.88  8.68  9.38  5.59  4.96  8.73  8.84  8.79 10.76  9.02  8.22  5.59  5.71  4.78  6.39 


table     1     1     1     1     2     2     2     2     3     3     3     3     4     4     4     4     5     5     5     5 

1       5.6  2.36 11.11 11.78 13.91 12.95 14.63 15.31  9.39  8.81 13.38 13.29 13.67 15.08 13.69 12.89 1